# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.21 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [2]:
from unityagents import UnityEnvironment
import numpy as np
import torch
from collections import deque

# please do not modify the line below
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [ 1.          0.          0.          0.          0.84408134  0.          0.
  1.          0.          0.0748472   0.          1.          0.          0.
  0.25755     1.          0.          0.          0.          0.74177343
  0.          1.          0.          0.          0.25854847  0.          0.
  1.          0.          0.09355672  0.          1.          0.          0.
  0.31969345  0.          0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


When finished, you can close the environment.

In [6]:
#env.close() #do not close when we are proceeding with training below
env_info = env.reset(train_mode=True)[brain_name] # reset the environment

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine! 

### agent configuration
Agent configuration is handled by a configuration dictionary to make it easier to do hyperparameter search. 
The config is passed as a "config" input parameter to the agent class.

In [7]:
from importlib import reload
import dqn_agent
import model
from datetime import datetime

agent_config = {
    "BUFFER_SIZE": int(1e5),  # replay buffer size
    "BATCH_SIZE": 32,         # minibatch size
    "GAMMA": 0.98,            # discount factor
    "TAU": 7e-4,              # for soft update of target parameters
    "LR": 4e-4,              # learning rate 
    "UPDATE_EVERY": 2,        # how often to update the network
    "N_LEARN_UPDATES": 1,     # How many iterations for training on the minibatch
    "hidden_layer_neurons": [128,128,128,128],   # Hidden layer neurons.
    "qnetwork_function":   model.QNetwork_with_connection_from_input   # qnetwork function.
}

agent = dqn_agent.Agent(state_size=state_size, 
                        action_size=action_size, 
                        seed=1235, 
                        config=agent_config
                       )
agent.agent_config

{'BUFFER_SIZE': 100000,
 'BATCH_SIZE': 32,
 'GAMMA': 0.98,
 'TAU': 0.0007,
 'LR': 0.0004,
 'UPDATE_EVERY': 2,
 'N_LEARN_UPDATES': 1,
 'hidden_layer_neurons': [128, 128, 128, 128],
 'qnetwork_function': model.QNetwork_with_connection_from_input}

In [8]:
# load the weights from file
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

#checkpoint = torch.load('checkpoint_2021-10-26T08 52 05.369710.pth', map_location=map_location)
#agent.qnetwork_local.load_state_dict(checkpoint)

In [9]:
from datetime import datetime

score_window_len=100
environment_solved_score = 13.0

def dqn(n_episodes: int=2400,
        eps_start: float=1.0,
        eps_end: float=0.01,
        eps_decay: float=0.996,
        N_REPEAT_ACTION: int=1):
    """Deep Q-Learning. Expects agent defined outside of function with name "agent"
    
    param: int n_episodes: maximum number of episodes
    param: float eps_start: start epsilon for greedy or exploratory action
    param: float eps_end: minimum value of epsilon
    param: float eps_decay: decay rate. eps_next = eps_current*eps_decay
    param: int N_REPEAT_ACTION: action repeat rate. Useful for less erratic motion
    returns: List[int]: List of scores.
    """
    
    scores = []                        # list containing scores from each episode
    n_actions = 0                      # number of actions taken
    scores_window = deque(maxlen=score_window_len)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]  
        score = 0
        while True:
            action = agent.act(state, eps)
            for n in range(N_REPEAT_ACTION):
                env_info = env.step(action)[brain_name]
                #n_actions += 1
                next_state = env_info.vector_observations[0]   # get the next state
                reward = env_info.rewards[0]                   # get the reward
                done = env_info.local_done[0]                  # see if episode has finished
                agent.step(state, action, reward, next_state, done)
                score += reward                                # update the score
                state = next_state    
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        ending = "\n" if i_episode % score_window_len == 0 else ""
        print((f'\rEpisode {int(i_episode):3d}'
               f'\tAv.Score: {np.mean(scores_window):5.2f}'
               f'\tCurrent: {int(scores_window[-1]):2d}'
               f"\tMax: {int(np.amax(scores_window)):2d}"
               f"\tMin: {int(np.amin(scores_window)):2d}"
               f"\tP25: {np.percentile(scores_window,25):3.1f}"
               f"\tP75: {np.percentile(scores_window,75):3.1f}"               
               f"\tMed: {int(np.median(scores_window)):2d}"
               f"\tstd: {np.std(scores_window):2.2f}"
               #f'\tNumber of actions: {int(n_actions):7d}'
               )
              , end=ending)
        if np.mean(scores_window)>=environment_solved_score:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), f'checkpoint_{datetime.now().isoformat()}.pth')
            break
    return scores

scores = dqn()
env.close()

Episode 100	Av.Score:  0.47	Current:  3	Max:  7	Min: -2	P25: -1.0	P75: 1.0	Med:  0	std: 1.66
Episode 200	Av.Score:  3.78	Current:  4	Max: 11	Min: -2	P25: 2.0	P75: 5.0	Med:  4	std: 2.546
Episode 300	Av.Score:  7.16	Current:  5	Max: 15	Min:  0	P25: 5.0	P75: 9.0	Med:  7	std: 2.92
Episode 400	Av.Score:  9.74	Current: 13	Max: 18	Min:  1	P25: 7.0	P75: 12.0	Med: 10	std: 3.06
Episode 500	Av.Score: 12.10	Current: 16	Max: 21	Min:  2	P25: 10.0	P75: 14.0	Med: 12	std: 3.54
Episode 600	Av.Score: 12.05	Current: 17	Max: 19	Min:  1	P25: 10.0	P75: 15.0	Med: 13	std: 3.74
Episode 668	Av.Score: 13.01	Current: 12	Max: 21	Min:  3	P25: 11.0	P75: 16.0	Med: 13	std: 3.65
Environment solved in 568 episodes!	Average Score: 13.01


### Plot scores
Plot the scores and the moving average as a function of episode.

In [10]:
import matplotlib.pyplot as plt
score_window_len=100
environment_solved_score = 13.0
# compute rolling average with a discrete convolution of ones (to make a sum)
# prepended with a list of 0s to account for initial episodes
rolling_average = np.convolve(
    np.ones(score_window_len),
    np.append(np.zeros(score_window_len-1),np.array(scores)), 
    'valid')/score_window_len

# plot the scores
np.zeros(100)
xs = np.arange(len(scores))
fig = plt.figure(figsize=[12,8])
ax = fig.add_subplot(111)
h1 = ax.plot(xs, scores, '.-')
h2 = ax.plot(xs, rolling_average)
h3 = ax.plot(xs, environment_solved_score*np.ones(len(scores)), ':')
ax.set_ylabel('Score')
ax.set_xlabel('Episode #')
ax.legend(["scores", "rolling average", "target score"])
fig.show()
fig.savefig("scores.png", dpi=400)

/opt/conda/lib/python3.6/site-packages/matplotlib/figure.py:418: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
